In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("../data/nyc_311_requests.csv")

/tmp/ipykernel_11511/473709468.py:1: DtypeWarning: Columns (39,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/nyc_311_requests.csv")


In [4]:
df.columns

Index(['unique_key', 'created_date', 'agency', 'agency_name', 'complaint_type',
       'descriptor', 'location_type', 'incident_zip', 'incident_address',
       'street_name', 'cross_street_1', 'cross_street_2', 'address_type',
       'city', 'status', 'resolution_description',
       'resolution_action_updated_date', 'community_board', 'bbl', 'borough',
       'x_coordinate_state_plane', 'y_coordinate_state_plane',
       'open_data_channel_type', 'park_facility_name', 'park_borough',
       'latitude', 'longitude', 'location', ':@computed_region_efsh_h5xi',
       ':@computed_region_f5dn_yrer', ':@computed_region_yeji_bk3q',
       ':@computed_region_92fq_4b7q', ':@computed_region_sbqj_enih',
       ':@computed_region_7mpf_4k6g', 'facility_type', 'intersection_street_1',
       'intersection_street_2', 'landmark', 'vehicle_type',
       'bridge_highway_name', 'bridge_highway_segment', 'closed_date',
       'taxi_pick_up_location', 'taxi_company_borough',
       'bridge_highway_direct

In [5]:
kept_columns = [
    'unique_key',
    'created_date',
    'closed_date',
    'agency',
    'agency_name',
    'complaint_type',
    'descriptor',
    'location_type',
    'incident_zip',
    'borough',
    'status',
    'latitude',
    'longitude']

clean_311 = df[kept_columns].copy()

In [6]:
clean_311.head()

,unique_key,created_date,closed_date,agency,agency_name,complaint_type,descriptor,location_type,incident_zip,borough,status,latitude,longitude
0,67002937,2025-12-01T02:53:00.000,NaN,DSNY,Department of Sanitation,Derelict Vehicles,Derelict Vehicles,Street,10467.0,BRONX,Open,40.875403,-73.876164
1,66999111,2025-12-01T02:24:58.000,NaN,DOT,Department of Transportation,Street Condition,Pothole,NaN,10301.0,STATEN ISLAND,Open,NaN,NaN
2,67006489,2025-12-01T02:05:37.000,NaN,NYPD,New York City Police Department,Noise - Residential,Banging/Pounding,Residential Building/House,11385.0,QUEENS,In Progress,40.702877,-73.904039
3,67005389,2025-12-01T02:05:05.000,NaN,NYPD,New York City Police Department,Noise - Street/Sidewalk,Loud Talking,Street/Sidewalk,11203.0,BROOKLYN,In Progress,40.651763,-73.944858
4,67002877,2025-12-01T02:03:03.000,NaN,NYPD,New York City Police Department,Illegal Parking,Double Parked Blocking Traffic,Street/Sidewalk,10467.0,BRONX,In Progress,40.858138,-73.865644


In [7]:
clean_311.isnull().sum()

unique_key            0
created_date          0
closed_date       12805
agency                0
agency_name           0
complaint_type        0
descriptor          722
location_type      3667
incident_zip        286
borough               0
status                0
latitude            347
longitude           347
dtype: int64

Checking for duplicates and filling in unknowns

In [8]:
clean_311.duplicated().sum()

np.int64(0)

In [9]:
clean_311['incident_zip'] = clean_311['incident_zip'].fillna('Unknown')

Extracting Datetime

In [10]:
clean_311['created_date'] = pd.to_datetime(clean_311['created_date'],errors='coerce')
clean_311['closed_date'] = pd.to_datetime(clean_311['closed_date'],errors='coerce')

Feature Engineering

In [14]:
clean_311['year'] = clean_311['created_date'].dt.year
clean_311['month'] = clean_311['created_date'].dt.month
clean_311['week_day'] = clean_311['created_date'].dt.day_name()
clean_311['hour'] = clean_311['created_date'].dt.hour
clean_311['resolution_time_hours'] = (clean_311['closed_date'] - clean_311['created_date']).dt.total_seconds() / 3600
clean_311['resolution_time_days'] = clean_311['resolution_time_hours'] / 24

In [15]:
clean_311.to_csv("../data/clean_nyc_311_requests.csv", index=False)